In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize

In [2]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/10_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_10ind = df.iloc[:1171].copy()
df_10ind['Date'] = pd.to_datetime(df_10ind['Date'], format='%Y%m')
df_10ind.set_index('Date', inplace=True)
df_10ind = df_10ind.apply(pd.to_numeric, errors='coerce')
df_10ind = df_10ind

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')

df_marketCap = df_numfirm * df_avgsize

In [4]:
def rolling_pf_SR(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x
        
    df = pf_weights.dropna()
        
    returns = df.loc[df.index]

    next_month_returns = returns.shift(-1)    

        
    df["Returns"] = (df * next_month_returns).sum(axis=1)


    return df.dropna()

In [5]:
rolling_pf_SR(df_10ind, 0.03, 60)

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1299/2668288717.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Returns"] = (df * next_month_returns).sum(axis=1)


,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Returns
Date,,,,,,,,,,,
1931-07-01,2.579031,-0.066929,-0.727597,-0.16063,0.915715,1.157005,-1.501925,1.30913,0.465788,-2.969588,20.864376
1931-08-01,2.450985,-0.233882,-0.804049,-0.044228,0.952139,1.161212,-1.241413,1.256903,0.230837,-2.728504,21.937025
1931-09-01,2.551207,-0.294499,-0.816239,0.017304,1.102745,1.342584,-1.295105,1.366894,0.306771,-3.281662,14606.278615
1931-10-01,371.472733,-718.975857,-994.646678,-1139.602694,775.596294,1672.28461,-660.912247,1498.521503,1119.04991,-1921.787575,19370119.400083
1931-11-01,478.12805,-708.019527,-1576.905031,-1347.331366,1096.279505,2128.844385,-770.015318,2125.655016,1557.218304,-2982.854016,13769181.08955
...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,1.127321,0.543369,0.535296,0.608261,3.499877,-2.069467,-2.86882,0.638505,0.722986,-1.737328,38.752273
2023-10-01,1.391259,0.80889,-0.776886,0.83889,4.663648,-2.996952,-3.990079,0.640201,1.219981,-0.798952,58.798312
2023-11-01,-2.175857,-0.039911,1.208255,1.417412,6.821198,-5.426306,-2.782664,0.838834,1.388513,-0.249474,123.362168


In [40]:
def calculate_portfolio_returns_next_month(portfolio_weights, portfolio_returns):
    # Assurez-vous que les index des deux DataFrames correspondent
    portfolio_returns = portfolio_weights.loc[portfolio_weights.index]

    # Décaler les rendements d'un mois
    next_month_returns = portfolio_returns.shift(-1)

    # Multipliez les poids par les rendements décalés et sommez-les pour obtenir le rendement total du portefeuille
    total_portfolio_returns_next_month = (portfolio_weights * next_month_returns).sum(axis=1)

    return total_portfolio_returns_next_month

# Utilisez la fonction pour calculer les rendements totaux du portefeuille avec les rendements du mois suivant
total_returns_next_month = calculate_portfolio_returns_next_month(rolling_pf_SR(df_10ind, 0.03, 60), df_10ind)


In [42]:
total_returns_next_month/100

Date
1931-07-01         0.208644
1931-08-01          0.21937
1931-09-01       146.062786
1931-10-01    193701.194001
1931-11-01    137691.810896
                  ...      
2023-09-01         0.387523
2023-10-01         0.587983
2023-11-01         1.233622
2023-12-01         0.573669
2024-01-01              0.0
Length: 1111, dtype: object

In [7]:
def rolling_pf_SR_NS(returns, rf, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def negativeSR(w):
            R = np.sum(z_bar * w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            SR = (R - rf) / V
            return -SR

        initial_weights = np.ones(num_assets) / num_assets
        bounds = [(0, None) for _ in range(num_assets)]
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(negativeSR, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()

In [8]:
rolling_pf_SR_NS(df_10ind, 0.03, 60)

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1931-07-01,0.0,0.0,0.0,0.0,0.0,0.522146,0.0,0.249004,0.228849,0.0
1931-08-01,0.0,0.0,0.0,0.0,0.0,0.440097,0.0,0.476103,0.0838,0.0
1931-09-01,0.0,0.0,0.0,0.0,0.0,0.459023,0.0,0.402502,0.138475,0.0
1931-10-01,0.0,0.0,0.0,0.0,0.0,0.742048,0.0,0.0,0.257952,0.0
1931-11-01,0.0,0.0,0.0,0.0,0.0,0.370176,0.0,0.109972,0.519851,0.0
...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.0,0.163154,0.0,0.015371,0.79313,0.0,0.0,0.0,0.028345,0.0
2023-10-01,0.0,0.276904,0.0,0.054216,0.668881,0.0,0.0,0.0,0.0,0.0
2023-11-01,0.0,0.027964,0.0,0.05476,0.917275,0.0,0.0,0.0,0.0,0.0


In [26]:
def rolling_pf_VAR(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        var = window.var()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / var) / np.sum(1 / var)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

In [27]:
def rolling_pf_VOL(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        # Calculer les poids en utilisant la formule
        std = window.std()  # Calcule l'écart-type de chaque actif sur la fenêtre
        weights = (1 / std) / np.sum(1 / std)  # Calculer les poids en utilisant la formule
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

In [28]:
def rolling_pf_SameWeights(returns, window_size):

    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        weights = (1 / window.shape[1])
        
        # Remplir les poids dans le DataFrame
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()

In [29]:
def rolling_pf_MarketCap(returns, window_size):

    pf_weights = pd.DataFrame(index=df_10ind.index, columns=df_10ind.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        weights = window / window.sum()
        
        pf_weights.iloc[i] = weights
        
    return pf_weights.dropna()  

In [30]:
#rolling_pf_MarketCap(df_10ind, 60)

In [31]:
def rolling_pf_minVAR(returns, window_size):
    num_assets = returns.shape[1]
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        
        z_bar = np.array(window.mean())
        sigma = window.cov()
        
        def min_volatility(w):
            w = np.array(w)
            V = np.sqrt(np.dot(w.T, np.dot(sigma, w)))
            return V

        initial_weights = np.ones(num_assets) / num_assets
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(min_volatility, initial_weights, method='SLSQP', constraints=constraints)
        pf_weights.iloc[i] = result.x

    return pf_weights.dropna()